In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
import io
sy.requires(SYFT_VERSION)
from syft import autocache

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

## Upload and retrieve SyftObject with blob storage (low level API)

In [ ]:
from syft.client.client import SyftClient
from syft.store.blob_storage import BlobDeposit
from syft.types.blob_storage import CreateBlobStorageEntry
from syft.types.syft_object import SyftObject

In [ ]:
def allocate_object(client: SyftClient, obj: SyftObject) -> BlobDeposit:
    create_blob_storage_entry = CreateBlobStorageEntry.from_obj(obj)
    return client.api.services.blob_storage.allocate(create_blob_storage_entry)

In [ ]:
user_object = domain_client.users[0]
user_object

In [ ]:
# write/deposit object
blob_deposit = allocate_object(domain_client, user_object)

In [ ]:
data = io.BytesIO(sy.serialize(user_object, to_bytes=True))
write_result = blob_deposit.write(data)
write_result

In [ ]:
# read/retrieve object
blob_retrieval = domain_client.api.services.blob_storage.read(blob_deposit.blob_storage_entry_id)
user_object_read = blob_retrieval.read()
user_object_read

In [ ]:
# delete object in blob storage
domain_client.api.services.blob_storage.delete(blob_deposit.blob_storage_entry_id)

## Upload and retrieve files with blob storage (low level API)

In [ ]:
from pathlib import Path

In [ ]:
def allocate_file(client: SyftClient, path: Path) -> BlobDeposit:
    create_blob_storage_entry = CreateBlobStorageEntry.from_path(path)
    return client.api.services.blob_storage.allocate(create_blob_storage_entry)


def upload_file(client: SyftClient, path: Path) -> sy.UID:
    blob_deposit = allocate_file(client, path)
    with open(path, "rb") as f:
        blob_deposit.write(f)
    return blob_deposit.blob_storage_entry_id


def retrieve_file(client, blob_storage_entry_id: sy.UID) -> Path:
    blob_retrieval = client.api.services.blob_storage.read(blob_storage_entry_id)
    file = blob_retrieval.read()
    return Path(file.file_name)

In [ ]:
data_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"
data_file = autocache(data_url, "csv")
data_file

In [ ]:
uploaded_file_storage_id = upload_file(domain_client, data_file)

In [ ]:
retrieved_file = retrieve_file(domain_client, uploaded_file_storage_id)

In [ ]:
import pandas as pd

Original file

In [ ]:
pd.read_csv(data_file, nrows=5)

Retrieved file

In [ ]:
pd.read_csv(retrieved_file, nrows=5)

In [ ]:
retrieved_file.unlink()

In [ ]:
# delete file from blob storage
domain_client.api.services.blob_storage.delete(uploaded_file_storage_id)

## From file using Action Object (Partial Functional)

In [ ]:
canada_dataset_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"
data_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"
data_file = autocache(data_url, "csv")
data_file

In [ ]:
# creating an action object from file
action_object = sy.ActionObject.from_file(filepath=str(data_file))

In [ ]:
data_ptr = action_object.send(domain_client)

In [ ]:
@sy.syft_function_single_use(trade_data=data_ptr)
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = pd.read_csv(data_ptr.syft_action_data.file_name, low_memory=False)
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

In [ ]:
sum_trade_value_mil(trade_data=data_ptr)

In [ ]:
# Cleanup local domain server
if node.node_type.value == "python":
    node.land()